In [ ]:
import pandas as pd
import os

pwd = os.getcwd()
filepath = os.path.join(pwd, "Placement_Data_Full_Class.csv")

data_import = pd.read_csv(filepath)
data_import

In [ ]:
data_import.describe()

In [ ]:
dropped_cols = data_import.copy()

dropped_cols = dropped_cols.drop(["sl_no", "salary"], axis=1)
dropped_cols

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

labeled_data = dropped_cols.copy()

cols_to_labelled = ["gender", "workex", "ssc_b", "hsc_b", "specialisation"]

labeled_data[cols_to_labelled] = labeled_data[cols_to_labelled].apply(le.fit_transform)

labeled_data

In [ ]:
hot_encoded_data = labeled_data.copy()

hot_encoded_data_temp = hot_encoded_data["status"]
hot_encoded_data = hot_encoded_data.drop("status", axis=1)

hot_encoded_data = pd.get_dummies(hot_encoded_data)

hot_encoded_data = pd.concat([hot_encoded_data, hot_encoded_data_temp], axis=1)
hot_encoded_data

In [ ]:
#scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_data = hot_encoded_data.copy()
labels = scaled_data.iloc[:, -1] #omit last col since those are our labels
scaled_data = scaled_data.iloc[:, :-1]

scaled_data = scaler.fit_transform(scaled_data)
scaled_data

In [ ]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
svm_clf = SVC()
rfc_clf = RandomForestClassifier()
vot_clf = VotingClassifier(estimators=[('lr', log_clf), ('sc', svm_clf), ('rf', rfc_clf)],voting="hard")

vot_clf.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rfc_clf, svm_clf, vot_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))